# Library

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [3]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [4]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
analysis_fold = 'analysis'

# load

In [5]:
# about 7m 39.0s
car = pd.read_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.28).csv'), low_memory=False)

In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25404385 entries, 0 to 25404384
Data columns (total 59 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [7]:
car.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

# 분석

In [8]:
car['등급'].value_counts(dropna=False)

2    11207677
3     8759393
1     2771380
4     1511211
5     1154724
Name: 등급, dtype: int64

In [9]:
car['연료'].value_counts(dropna=False)

휘발유            12101949
경유              9860384
LPG(액화석유가스)     1896593
휘발유 하이브리드       1071363
전기               366804
경유 하이브리드          34191
CNG(압축천연가스)       30311
수소                28102
LPG 하이브리드         14350
CNG 하이브리드           304
LNG(액화천연가스)          25
무동력                   7
알코올                   2
Name: 연료, dtype: int64

## 전기차와 하이브리드

In [10]:
df = car.loc[(car['연료'] == '전기') | (car['연료'] == '휘발유 하이브리드') | (car['연료'] == '경유 하이브리드') | (car['연료'] == 'LPG 하이브리드') | (car['연료'] == 'CNG 하이브리드')].reset_index(drop=True)
df.shape

(1487012, 59)

In [11]:
stat = df.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
stat

대기관리권역YN         N         Y
대기관리권역 시도                   
그 외    강원 40664.00       NaN
       제주 69786.00       NaN
남부권    광주      NaN  37118.00
       전남 33388.00  27301.00
동남권    경남 20457.00  70374.00
       경북 20845.00  39861.00
       대구      NaN  80808.00
       부산      NaN  97512.00
       울산      NaN  27683.00
수도권    경기  5985.00 358227.00
       서울      NaN 222407.00
       인천   202.00 136998.00
중부권    대전      NaN  41298.00
       세종      NaN  15124.00
       전북 12615.00  29742.00
       충남  1062.00  53035.00
       충북  4594.00  39926.00

## 전기차

In [12]:
ev = df.loc[df['연료'] == '전기'].reset_index(drop=True)
ev.shape

(366804, 59)

In [13]:
ev_stat = ev.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
ev_stat

대기관리권역YN         N        Y
대기관리권역 시도                  
그 외    강원 12991.00      NaN
       제주 31649.00      NaN
남부권    광주      NaN  8458.00
       전남  6265.00  8288.00
동남권    경남  5552.00 15379.00
       경북  7314.00 10568.00
       대구      NaN 23030.00
       부산      NaN 20517.00
       울산      NaN  4764.00
수도권    경기  1835.00 69655.00
       서울      NaN 57819.00
       인천    50.00 23703.00
중부권    대전      NaN 13417.00
       세종      NaN  2783.00
       전북  4770.00  7642.00
       충남   424.00 15571.00
       충북  1815.00 12545.00

## 하이브리드

In [14]:
hybrid = df.loc[(df['연료'] == '휘발유 하이브리드') | (df['연료'] == '경유 하이브리드') | (df['연료'] == 'LPG 하이브리드') | (df['연료'] == 'CNG 하이브리드')].reset_index(drop=True)
hybrid.shape

(1120208, 59)

In [15]:
hybrid_stat = hybrid.groupby(['대기관리권역', '시도', '대기관리권역YN'], dropna=False)['차대번호'].count().unstack()
hybrid_stat

대기관리권역YN         N         Y
대기관리권역 시도                   
그 외    강원 27673.00       NaN
       제주 38137.00       NaN
남부권    광주      NaN  28660.00
       전남 27123.00  19013.00
동남권    경남 14905.00  54995.00
       경북 13531.00  29293.00
       대구      NaN  57778.00
       부산      NaN  76995.00
       울산      NaN  22919.00
수도권    경기  4150.00 288572.00
       서울      NaN 164588.00
       인천   152.00 113295.00
중부권    대전      NaN  27881.00
       세종      NaN  12341.00
       전북  7845.00  22100.00
       충남   638.00  37464.00
       충북  2779.00  27381.00

## 출력

In [18]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.05.02_PM.04.34.40'

In [19]:
with pd.ExcelWriter(os.path.join(analysis_fold, f'통계_전기차_하이브리드_대기관리권역별{today_date}.xlsx')) as writer:
    stat.to_excel(writer, sheet_name='전기_하이브리드')
    ev_stat.to_excel(writer, sheet_name='전기차')
    hybrid_stat.to_excel(writer, sheet_name='하이브리드')

# 코드 마지막